In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件，跟line溝通用
line_bot_api = LineBotApi("your channel access token")
# 收發消息專用
handler = WebhookHandler("your channel secret")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''

from linebot.models import(
    MessageEvent, ImageMessage, VideoMessage, AudioMessage, TextSendMessage
)

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    # 回應圖片上傳的id
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id))
   
    # 依據消息的id，向line索取檔案回來
    message_content = line_bot_api.get_message_content(event.message.id)
    # 存在本地檔，以圖片的方式儲存
    with open(event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [ ]:
'''
影片消息，留給你
'''

@handler.add(MessageEvent, message=VideoMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Video has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.mp4', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [ ]:
'''
音訊消息，留給你
'''

@handler.add(MessageEvent, message=AudioMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Audio has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.mp3', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [ ]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1f6c-34-73-78-238.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"image","id":"16910580683273","contentProvider":{"type":"line"}},"webhookEventId":"01GERN9308R9E0BF1SFBF5F8M3","deliveryContext":{"isRedelivery":false},"timestamp":1665127451213,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"fdc8fd4b93d54be8b8c6fc70e641edfb","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 07:24:14] "POST / HTTP/1.1" 200 -


{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"audio","id":"16910583581082","duration":3452,"contentProvider":{"type":"line"}},"webhookEventId":"01GERNA7BNNWQFJB9KRJ1GDYTT","deliveryContext":{"isRedelivery":false},"timestamp":1665127488653,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"390b34f5e5b644458b67ce65d82378ff","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 07:24:51] "POST / HTTP/1.1" 200 -


{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"video","id":"16910587332437","duration":16732,"contentProvider":{"type":"line"}},"webhookEventId":"01GERNBX8HN5549BJ8J30Z5CFB","deliveryContext":{"isRedelivery":false},"timestamp":1665127543578,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"8de35bb3e56e45679e05a3edec0c7085","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 07:25:47] "POST / HTTP/1.1" 200 -
